<a href="https://colab.research.google.com/github/abdelilah-bouslama/Data-Science/blob/master/Apache_Spark/cca175/DataFrame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
!wget -q https://downloads.apache.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

In [0]:
!tar xf spark-2.4.5-bin-hadoop2.7.tgz

In [0]:
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

# **DATAFRAME from CSV FILE**

In [0]:
ordersDF = spark.read.csv("/content/drive/My Drive/BIG DATA/pyspark/CCA-175/dataset/retail_db/orders/part-00000", sep=",")

In [0]:
ordersDF.first()

Row(_c0='1', _c1='2013-07-25 00:00:00.0', _c2='11599', _c3='CLOSED')

In [0]:
ordersDF.printSchema()
ordersDF.count()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)



68883

In [0]:
ordersDF = spark.read.csv("/content/drive/My Drive/BIG DATA/pyspark/CCA-175/dataset/retail_db/orders/part-00000", sep=",", schema='order_id int, order_date string, order_customer_id int, order_status string')

In [0]:
ordersDF.printSchema()

In [0]:
#Another way to define schema
ordersDF = spark.read.csv("/content/drive/My Drive/BIG DATA/pyspark/CCA-175/dataset/retail_db/orders/part-00000", sep=",")

In [0]:
ordersDF = ordersDF.toDF('order_id int', 'order_date string', 'order_customer_id int', 'order_status string')
ordersDF.printSchema()

In [0]:
# load CSV using format
ordersDF = spark.read.format('csv') \
    .option('sep', ',') \
    .schema('order_id int, order_date string, order_customer_id int, order_status string') \
    .load("/content/drive/My Drive/BIG DATA/pyspark/CCA-175/dataset/retail_db/orders/part-00000")

In [0]:
ordersDF.printSchema()

In [0]:
#Second way to define dataType using casting
ordersDF = spark.read.csv("/content/drive/My Drive/BIG DATA/pyspark/CCA-175/dataset/retail_db/orders/part-00000", sep=",").toDF('order_id', 'order_date', 'order_customer_id', 'order_status')

In [0]:
ordersDF.printSchema()

In [0]:
from pyspark.sql.types import IntegerType
orders = ordersDF.select(ordersDF.order_id.cast("int"), ordersDF.order_date, ordersDF.order_customer_id.cast(IntegerType()), ordersDF.order_status)

In [0]:
orders

In [0]:
#Third way to scpecify dataType
orders = ordersDF.withColumn('order_id', ordersDF.order_id.cast("int")) \
    .withColumn('order_customer_id', ordersDF.order_customer_id.cast(IntegerType()))
orders

DataFrame[order_id: int, order_date: string, order_customer_id: int, order_status: string]

In [0]:
#Agg functions on orders DataFrame
orders.select('order_status').distinct().count() #count
from pyspark.sql.functions import countDistinct
orders.select(countDistinct(orders.order_status)).show()

#spark.sql("show functions").where('function like "%stinct%"').show()

+----------------------------+
|count(DISTINCT order_status)|
+----------------------------+
|                           9|
+----------------------------+



# **JOIN Between DATAFRAME**

In [0]:
ordersDF = spark.read.csv("/content/drive/My Drive/BIG DATA/pyspark/CCA-175/dataset/retail_db/orders/part-00000", sep=",", schema='order_id int, order_date string, order_customer_id int, order_status string')

In [0]:
ordersDF.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_customer_id: integer (nullable = true)
 |-- order_status: string (nullable = true)



In [0]:
ordersDF.select('order_id', ordersDF.order_date, 'order_customer_id').show(truncate = False) # ordersDF.select(ordersDF.order_id, ordersDF.order_customer_id).show()

+--------+---------------------+-----------------+
|order_id|order_date           |order_customer_id|
+--------+---------------------+-----------------+
|1       |2013-07-25 00:00:00.0|11599            |
|2       |2013-07-25 00:00:00.0|256              |
|3       |2013-07-25 00:00:00.0|12111            |
|4       |2013-07-25 00:00:00.0|8827             |
|5       |2013-07-25 00:00:00.0|11318            |
|6       |2013-07-25 00:00:00.0|7130             |
|7       |2013-07-25 00:00:00.0|4530             |
|8       |2013-07-25 00:00:00.0|2911             |
|9       |2013-07-25 00:00:00.0|5657             |
|10      |2013-07-25 00:00:00.0|5648             |
|11      |2013-07-25 00:00:00.0|918              |
|12      |2013-07-25 00:00:00.0|1837             |
|13      |2013-07-25 00:00:00.0|9149             |
|14      |2013-07-25 00:00:00.0|9842             |
|15      |2013-07-25 00:00:00.0|2568             |
|16      |2013-07-25 00:00:00.0|7276             |
|17      |2013-07-25 00:00:00.0

In [0]:
# get Y-month of orders 
from pyspark.sql.functions import substring
ordersDF.select('order_id',substring(ordersDF.order_date, 1, 7).alias('order_date')).show()

+--------+----------+
|order_id|order_date|
+--------+----------+
|       1|   2013-07|
|       2|   2013-07|
|       3|   2013-07|
|       4|   2013-07|
|       5|   2013-07|
|       6|   2013-07|
|       7|   2013-07|
|       8|   2013-07|
|       9|   2013-07|
|      10|   2013-07|
|      11|   2013-07|
|      12|   2013-07|
|      13|   2013-07|
|      14|   2013-07|
|      15|   2013-07|
|      16|   2013-07|
|      17|   2013-07|
|      18|   2013-07|
|      19|   2013-07|
|      20|   2013-07|
+--------+----------+
only showing top 20 rows



In [0]:
#get All columns with transform format for particular column
ordersDF.withColumn('order_id', ordersDF.order_id).withColumn('order_date', substring(ordersDF.order_date, 1, 7)).show() #use select('order_id', 'order_date') to filter

+--------+----------+-----------------+---------------+
|order_id|order_date|order_customer_id|   order_status|
+--------+----------+-----------------+---------------+
|       1|   2013-07|            11599|         CLOSED|
|       2|   2013-07|              256|PENDING_PAYMENT|
|       3|   2013-07|            12111|       COMPLETE|
|       4|   2013-07|             8827|         CLOSED|
|       5|   2013-07|            11318|       COMPLETE|
|       6|   2013-07|             7130|       COMPLETE|
|       7|   2013-07|             4530|       COMPLETE|
|       8|   2013-07|             2911|     PROCESSING|
|       9|   2013-07|             5657|PENDING_PAYMENT|
|      10|   2013-07|             5648|PENDING_PAYMENT|
|      11|   2013-07|              918| PAYMENT_REVIEW|
|      12|   2013-07|             1837|         CLOSED|
|      13|   2013-07|             9149|PENDING_PAYMENT|
|      14|   2013-07|             9842|     PROCESSING|
|      15|   2013-07|             2568|       CO

In [0]:
# drop columns
ordersDF.drop('order_id').show(truncate = False)

+---------------------+-----------------+---------------+
|order_date           |order_customer_id|order_status   |
+---------------------+-----------------+---------------+
|2013-07-25 00:00:00.0|11599            |CLOSED         |
|2013-07-25 00:00:00.0|256              |PENDING_PAYMENT|
|2013-07-25 00:00:00.0|12111            |COMPLETE       |
|2013-07-25 00:00:00.0|8827             |CLOSED         |
|2013-07-25 00:00:00.0|11318            |COMPLETE       |
|2013-07-25 00:00:00.0|7130             |COMPLETE       |
|2013-07-25 00:00:00.0|4530             |COMPLETE       |
|2013-07-25 00:00:00.0|2911             |PROCESSING     |
|2013-07-25 00:00:00.0|5657             |PENDING_PAYMENT|
|2013-07-25 00:00:00.0|5648             |PENDING_PAYMENT|
|2013-07-25 00:00:00.0|918              |PAYMENT_REVIEW |
|2013-07-25 00:00:00.0|1837             |CLOSED         |
|2013-07-25 00:00:00.0|9149             |PENDING_PAYMENT|
|2013-07-25 00:00:00.0|9842             |PROCESSING     |
|2013-07-25 00

In [0]:
# using Sql style with transform actions
ordersDF.selectExpr('order_id','substring(order_date, 1, 7) as order_date').show()

+--------+----------+
|order_id|order_date|
+--------+----------+
|       1|   2013-07|
|       2|   2013-07|
|       3|   2013-07|
|       4|   2013-07|
|       5|   2013-07|
|       6|   2013-07|
|       7|   2013-07|
|       8|   2013-07|
|       9|   2013-07|
|      10|   2013-07|
|      11|   2013-07|
|      12|   2013-07|
|      13|   2013-07|
|      14|   2013-07|
|      15|   2013-07|
|      16|   2013-07|
|      17|   2013-07|
|      18|   2013-07|
|      19|   2013-07|
|      20|   2013-07|
+--------+----------+
only showing top 20 rows



In [0]:
# filter data using filter function
ordersDF.selectExpr('order_id', 'order_date', 'order_status').filter('order_date > "2013-07-01 00:00:00" and order_status in("COMPLETE")').show(truncate=False)
#OR
ordersDF.selectExpr('order_id', 'order_date').filter(ordersDF.order_date > "2013-07-01 00:00:00").show(10,truncate=False)
#Another example using In

+--------+---------------------+------------+
|order_id|order_date           |order_status|
+--------+---------------------+------------+
|3       |2013-07-25 00:00:00.0|COMPLETE    |
|5       |2013-07-25 00:00:00.0|COMPLETE    |
|6       |2013-07-25 00:00:00.0|COMPLETE    |
|7       |2013-07-25 00:00:00.0|COMPLETE    |
|15      |2013-07-25 00:00:00.0|COMPLETE    |
|17      |2013-07-25 00:00:00.0|COMPLETE    |
|22      |2013-07-25 00:00:00.0|COMPLETE    |
|26      |2013-07-25 00:00:00.0|COMPLETE    |
|28      |2013-07-25 00:00:00.0|COMPLETE    |
|32      |2013-07-25 00:00:00.0|COMPLETE    |
|35      |2013-07-25 00:00:00.0|COMPLETE    |
|45      |2013-07-25 00:00:00.0|COMPLETE    |
|56      |2013-07-25 00:00:00.0|COMPLETE    |
|63      |2013-07-25 00:00:00.0|COMPLETE    |
|65      |2013-07-25 00:00:00.0|COMPLETE    |
|67      |2013-07-25 00:00:00.0|COMPLETE    |
|71      |2013-07-25 00:00:00.0|COMPLETE    |
|72      |2013-07-25 00:00:00.0|COMPLETE    |
|76      |2013-07-25 00:00:00.0|CO

In [0]:
help(ordersDF.sort)

Help on method sort in module pyspark.sql.dataframe:

sort(*cols, **kwargs) method of pyspark.sql.dataframe.DataFrame instance
    Returns a new :class:`DataFrame` sorted by the specified column(s).
    
    :param cols: list of :class:`Column` or column names to sort by.
    :param ascending: boolean or list of boolean (default ``True``).
        Sort ascending vs. descending. Specify list for multiple sort orders.
        If a list is specified, length of the list must equal length of the `cols`.
    
    >>> df.sort(df.age.desc()).collect()
    [Row(age=5, name='Bob'), Row(age=2, name='Alice')]
    >>> df.sort("age", ascending=False).collect()
    [Row(age=5, name='Bob'), Row(age=2, name='Alice')]
    >>> df.orderBy(df.age.desc()).collect()
    [Row(age=5, name='Bob'), Row(age=2, name='Alice')]
    >>> from pyspark.sql.functions import *
    >>> df.sort(asc("age")).collect()
    [Row(age=2, name='Alice'), Row(age=5, name='Bob')]
    >>> df.orderBy(desc("age"), "name").collect()
    

In [0]:
#Get orders which are either COMPLETE or CLOSED, and sorted base on order_date
ordersDF.filter('order_status in ("COMPLETE", "CLOSED")') \
    .sort('order_date',ascending=False) \
    .show()
# another example .sort(ordersDF.order_customer_id.desc())\

+--------+--------------------+-----------------+------------+
|order_id|          order_date|order_customer_id|order_status|
+--------+--------------------+-----------------+------------+
|   57595|2014-07-24 00:00:...|             9102|    COMPLETE|
|   57647|2014-07-24 00:00:...|            11856|    COMPLETE|
|   57603|2014-07-24 00:00:...|            10713|    COMPLETE|
|   57606|2014-07-24 00:00:...|             2633|    COMPLETE|
|   57607|2014-07-24 00:00:...|             2666|      CLOSED|
|   57612|2014-07-24 00:00:...|             3199|    COMPLETE|
|   57613|2014-07-24 00:00:...|             9278|      CLOSED|
|   57616|2014-07-24 00:00:...|             6545|      CLOSED|
|   57617|2014-07-24 00:00:...|                3|    COMPLETE|
|   57619|2014-07-24 00:00:...|            11662|    COMPLETE|
|   57622|2014-07-24 00:00:...|             3683|    COMPLETE|
|   57623|2014-07-24 00:00:...|             5049|    COMPLETE|
|   57627|2014-07-24 00:00:...|            11707|    CO

In [0]:
#Get orders which are either COMPLETE or CLOSED and placed in the month of 2013/08
ordersDF.filter('order_status in("COMPLETE", "CLOSED")') \
    .where('substring(order_date,0,7) == "2013-08"') \
    .show()
#Another way
# can replace where with & in filter function
ordersDF.filter((ordersDF.order_status == "COMPLETE") | (ordersDF.order_status =="CLOSED")) \
    .where('substring(order_date,0,7) == "2013-08"') \
    .show()

#Another way
ordersDF.filter(ordersDF.order_status.isin("COMPLETE", "CLOSED")) \
    .where('substring(order_date,0,7) == "2013-08"') \
    .show()

+--------+--------------------+-----------------+------------+
|order_id|          order_date|order_customer_id|order_status|
+--------+--------------------+-----------------+------------+
|    1297|2013-08-01 00:00:...|            11607|    COMPLETE|
|    1298|2013-08-01 00:00:...|             5105|      CLOSED|
|    1299|2013-08-01 00:00:...|             7802|    COMPLETE|
|    1302|2013-08-01 00:00:...|             1695|    COMPLETE|
|    1304|2013-08-01 00:00:...|             2059|    COMPLETE|
|    1305|2013-08-01 00:00:...|             3844|    COMPLETE|
|    1307|2013-08-01 00:00:...|             4474|    COMPLETE|
|    1309|2013-08-01 00:00:...|             2367|      CLOSED|
|    1312|2013-08-01 00:00:...|            12291|    COMPLETE|
|    1314|2013-08-01 00:00:...|            10993|    COMPLETE|
|    1315|2013-08-01 00:00:...|             5660|    COMPLETE|
|    1318|2013-08-01 00:00:...|             4212|    COMPLETE|
|    1319|2013-08-01 00:00:...|             3966|    CO

In [0]:
# using MAGIC functions(__or__, __and__)
ordersDF.filter((ordersDF.order_status == "COMPLETE").__or__(ordersDF.order_status =="CLOSED")) \
    .where('substring(order_date,0,7) like "2013-08%"') \
    .show()
#use dataFrame Like function
from pyspark.sql.functions import substring
ordersDF.filter((ordersDF.order_status == "COMPLETE").__or__(ordersDF.order_status =="CLOSED")) \
    .where(substring(ordersDF.order_date, 0, 7).like("2013-08%")) \
    .show()

+--------+--------------------+-----------------+------------+
|order_id|          order_date|order_customer_id|order_status|
+--------+--------------------+-----------------+------------+
|    1297|2013-08-01 00:00:...|            11607|    COMPLETE|
|    1298|2013-08-01 00:00:...|             5105|      CLOSED|
|    1299|2013-08-01 00:00:...|             7802|    COMPLETE|
|    1302|2013-08-01 00:00:...|             1695|    COMPLETE|
|    1304|2013-08-01 00:00:...|             2059|    COMPLETE|
|    1305|2013-08-01 00:00:...|             3844|    COMPLETE|
|    1307|2013-08-01 00:00:...|             4474|    COMPLETE|
|    1309|2013-08-01 00:00:...|             2367|      CLOSED|
|    1312|2013-08-01 00:00:...|            12291|    COMPLETE|
|    1314|2013-08-01 00:00:...|            10993|    COMPLETE|
|    1315|2013-08-01 00:00:...|             5660|    COMPLETE|
|    1318|2013-08-01 00:00:...|             4212|    COMPLETE|
|    1319|2013-08-01 00:00:...|             3966|    CO

In [0]:
#Get all the orders which are placed on first of every month
#ordersDF.show(truncate=False)
from pyspark.sql.functions import dayofmonth
ordersDF.filter(dayofmonth(ordersDF.order_date) == '01').show(2,truncate=False)
#Another way using sql
ordersDF.filter("dayofmonth(order_date) == '01'").show(2,truncate=False)

+--------+---------------------+-----------------+------------+
|order_id|order_date           |order_customer_id|order_status|
+--------+---------------------+-----------------+------------+
|1297    |2013-08-01 00:00:00.0|11607            |COMPLETE    |
|1298    |2013-08-01 00:00:00.0|5105             |CLOSED      |
+--------+---------------------+-----------------+------------+
only showing top 2 rows

+--------+---------------------+-----------------+------------+
|order_id|order_date           |order_customer_id|order_status|
+--------+---------------------+-----------------+------------+
|1297    |2013-08-01 00:00:00.0|11607            |COMPLETE    |
|1298    |2013-08-01 00:00:00.0|5105             |CLOSED      |
+--------+---------------------+-----------------+------------+
only showing top 2 rows



In [0]:
order_itemsDF = spark.read.csv("/content/drive/My Drive/BIG DATA/pyspark/CCA-175/dataset/retail_db/order_items/part-00000", sep=',', schema='order_item_id int, order_item_order_id int, order_item_product_id int, order_item_quantity int, order_item_subtotal float, order_item_product_price float')

In [0]:
order_itemsDF.printSchema()

root
 |-- order_item_id: integer (nullable = true)
 |-- order_item_order_id: integer (nullable = true)
 |-- order_item_product_id: integer (nullable = true)
 |-- order_item_quantity: integer (nullable = true)
 |-- order_item_subtotal: float (nullable = true)
 |-- order_item_product_price: float (nullable = true)



In [0]:
#Get order_items where order_item_subtotal is not equal to the product of order_item_quantity and order_item_product_price
from pyspark.sql.functions import round 
order_itemsDF.select('order_item_subtotal', 'order_item_quantity', 'order_item_product_price').filter(order_itemsDF.order_item_subtotal != round((order_itemsDF.order_item_product_price *  order_itemsDF.order_item_quantity), 2)) \
    .withColumn('product_price_per_quanity', round((order_itemsDF.order_item_product_price *  order_itemsDF.order_item_quantity), 2)) \
    .show()

+-------------------+-------------------+------------------------+-------------------------+
|order_item_subtotal|order_item_quantity|order_item_product_price|product_price_per_quanity|
+-------------------+-------------------+------------------------+-------------------------+
+-------------------+-------------------+------------------------+-------------------------+



In [0]:
help(order_itemsDF.join)

Help on method join in module pyspark.sql.dataframe:

join(other, on=None, how=None) method of pyspark.sql.dataframe.DataFrame instance
    Joins with another :class:`DataFrame`, using the given join expression.
    
    :param other: Right side of the join
    :param on: a string for the join column name, a list of column names,
        a join expression (Column), or a list of Columns.
        If `on` is a string or a list of strings indicating the name of the join column(s),
        the column(s) must exist on both sides, and this performs an equi-join.
    :param how: str, default ``inner``. Must be one of: ``inner``, ``cross``, ``outer``,
        ``full``, ``full_outer``, ``left``, ``left_outer``, ``right``, ``right_outer``,
        ``left_semi``, and ``left_anti``.
    
    The following performs a full outer join between ``df1`` and ``df2``.
    
    >>> df.join(df2, df.name == df2.name, 'outer').select(df.name, df2.height).collect()
    [Row(name=None, height=80), Row(name='Bob'

In [0]:
#Get all order_items corresponding to COMPLETE and CLOSED orders
orderItemsCompleteClosed =  order_itemsDF.select(order_itemsDF.order_item_order_id)\
    .join(ordersDF, on = order_itemsDF.order_item_order_id == ordersDF.order_id, how='inner')\
    .filter(ordersDF.order_status.isin('COMPLETE', 'CLOSED'))\
    .show()

+-------------------+--------+--------------------+-----------------+------------+
|order_item_order_id|order_id|          order_date|order_customer_id|order_status|
+-------------------+--------+--------------------+-----------------+------------+
|                  1|       1|2013-07-25 00:00:...|            11599|      CLOSED|
|                  4|       4|2013-07-25 00:00:...|             8827|      CLOSED|
|                  4|       4|2013-07-25 00:00:...|             8827|      CLOSED|
|                  4|       4|2013-07-25 00:00:...|             8827|      CLOSED|
|                  4|       4|2013-07-25 00:00:...|             8827|      CLOSED|
|                  5|       5|2013-07-25 00:00:...|            11318|    COMPLETE|
|                  5|       5|2013-07-25 00:00:...|            11318|    COMPLETE|
|                  5|       5|2013-07-25 00:00:...|            11318|    COMPLETE|
|                  5|       5|2013-07-25 00:00:...|            11318|    COMPLETE|
|   

In [0]:
#Get all orders where there is no corresponding order_items
result = ordersDF.join(order_itemsDF, on = ordersDF.order_id == order_itemsDF.order_item_order_id, how='left_outer')\
    .filter('order_item_order_id is null')\
    .show()

+--------+--------------------+-----------------+---------------+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_id|          order_date|order_customer_id|   order_status|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+--------+--------------------+-----------------+---------------+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|         null|               null|                 null|               null|               null|                    null|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|         null|               null|                 null|               null|               null|                    null|
|      22|2013-07-25 00:00:...|              333|       COMPLETE|

In [0]:
# check if there are any order_items where there is no corresponding order in the orders data set
result = ordersDF.join(order_itemsDF, on = ordersDF.order_id == order_itemsDF.order_item_order_id, how='right_outer')\
    .filter('order_id is null')\
    .show()
order_itemsDF.filter('order_item_order_id is null').show()

+--------+----------+-----------------+------------+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_id|order_date|order_customer_id|order_status|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+--------+----------+-----------------+------------+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
+--------+----------+-----------------+------------+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+

In [0]:
#Agg functions on orerItems DataFrame
from pyspark.sql.functions import round
orderItemSubTotals  = order_itemsDF.agg({"order_item_subtotal": "avg"})\
    .select(round('avg(order_item_subtotal)', 2).alias('avg_item_subtotal'))
orderItemSubTotals.show()

+-----------------+
|avg_item_subtotal|
+-----------------+
|           199.32|
+-----------------+



In [0]:
help(order_itemsDF.groupBy)

Help on method groupBy in module pyspark.sql.dataframe:

groupBy(*cols) method of pyspark.sql.dataframe.DataFrame instance
    Groups the :class:`DataFrame` using the specified columns,
    so we can run aggregation on them. See :class:`GroupedData`
    for all the available aggregate functions.
    
    :func:`groupby` is an alias for :func:`groupBy`.
    
    :param cols: list of columns to group by.
        Each element should be a column name (string) or an expression (:class:`Column`).
    
    >>> df.groupBy().avg().collect()
    [Row(avg(age)=3.5)]
    >>> sorted(df.groupBy('name').agg({'age': 'mean'}).collect())
    [Row(name='Alice', avg(age)=2.0), Row(name='Bob', avg(age)=5.0)]
    >>> sorted(df.groupBy(df.name).avg().collect())
    [Row(name='Alice', avg(age)=2.0), Row(name='Bob', avg(age)=5.0)]
    >>> sorted(df.groupBy(['name', df.age]).count().collect())
    [Row(name='Alice', age=2, count=1), Row(name='Bob', age=5, count=1)]
    
    .. versionadded:: 1.3



In [0]:
#Get revenue of each order_id
from pyspark.sql.functions import sum
#order_itemsDF.agg({order_itemsDF.order_item_subtotal, "avg"})\
#    .groupBy(order_itemsDF.order_item_order_id)\
#    .show()
#Important : see video
order_itemsDF.groupBy('order_item_order_id')\
    .agg(sum('order_item_subtotal').alias('order_revenue'))\
    .show()

+-------------------+------------------+
|order_item_order_id|     order_revenue|
+-------------------+------------------+
|                148|479.99000549316406|
|                463| 829.9200096130371|
|                471|169.98000717163086|
|                496|  441.950008392334|
|               1088|249.97000885009766|
|               1580|299.95001220703125|
|               1591| 439.8599967956543|
|               1645| 1509.790023803711|
|               2366| 299.9700012207031|
|               2659| 724.9100151062012|
|               2866|  569.960018157959|
|               3175|209.97000122070312|
|               3749|143.97000122070312|
|               3794|299.95001220703125|
|               3918| 829.9300155639648|
|               3997| 579.9500122070312|
|               4101|129.99000549316406|
|               4519|  79.9800033569336|
|               4818| 399.9800109863281|
|               4900| 179.9700050354004|
+-------------------+------------------+
only showing top

In [0]:
#Get count by status from orders
from pyspark.sql.functions import count
ordersDF.groupBy('order_status')\
    .agg(count('order_status').alias('order_per_status'))\
    .show()

+---------------+----------------+
|   order_status|order_per_status|
+---------------+----------------+
|PENDING_PAYMENT|           15030|
|       COMPLETE|           22899|
|        ON_HOLD|            3798|
| PAYMENT_REVIEW|             729|
|     PROCESSING|            8275|
|         CLOSED|            7556|
|SUSPECTED_FRAUD|            1558|
|        PENDING|            7610|
|       CANCELED|            1428|
+---------------+----------------+



In [0]:
#Get daily product Revenue
ordersDF.show(truncate=False)
order_itemsDF.show(truncate=False)

+--------+---------------------+-----------------+---------------+
|order_id|order_date           |order_customer_id|order_status   |
+--------+---------------------+-----------------+---------------+
|1       |2013-07-25 00:00:00.0|11599            |CLOSED         |
|2       |2013-07-25 00:00:00.0|256              |PENDING_PAYMENT|
|3       |2013-07-25 00:00:00.0|12111            |COMPLETE       |
|4       |2013-07-25 00:00:00.0|8827             |CLOSED         |
|5       |2013-07-25 00:00:00.0|11318            |COMPLETE       |
|6       |2013-07-25 00:00:00.0|7130             |COMPLETE       |
|7       |2013-07-25 00:00:00.0|4530             |COMPLETE       |
|8       |2013-07-25 00:00:00.0|2911             |PROCESSING     |
|9       |2013-07-25 00:00:00.0|5657             |PENDING_PAYMENT|
|10      |2013-07-25 00:00:00.0|5648             |PENDING_PAYMENT|
|11      |2013-07-25 00:00:00.0|918              |PAYMENT_REVIEW |
|12      |2013-07-25 00:00:00.0|1837             |CLOSED      

In [0]:
order_itemsDF.join(ordersDF, on=order_itemsDF.order_item_order_id == ordersDF.order_id,how='inner')\
    .groupBy(ordersDF.order_date, order_itemsDF.order_item_product_id)\
    .agg(round(sum(order_itemsDF.order_item_subtotal),2).alias('daily_product_revenue'), count(order_itemsDF.order_item_id).alias('order_item_count'))\
    .select(order_itemsDF.order_item_product_id, ordersDF.order_date, 'daily_product_revenue', 'order_item_count')\
    .sort(['order_date', 'daily_product_revenue'], ascending=[False, True])\
    .show(truncate=False)
#can also order by .sort('order_date', order_itemsDF.order_item_product_id.desc())

+---------------------+---------------------+---------------------+----------------+
|order_item_product_id|order_date           |daily_product_revenue|order_item_count|
+---------------------+---------------------+---------------------+----------------+
|793                  |2014-07-24 00:00:00.0|14.99                |1               |
|886                  |2014-07-24 00:00:00.0|24.99                |1               |
|905                  |2014-07-24 00:00:00.0|24.99                |1               |
|835                  |2014-07-24 00:00:00.0|31.99                |1               |
|703                  |2014-07-24 00:00:00.0|39.98                |1               |
|917                  |2014-07-24 00:00:00.0|43.98                |1               |
|134                  |2014-07-24 00:00:00.0|50.0                 |1               |
|797                  |2014-07-24 00:00:00.0|53.97                |2               |
|792                  |2014-07-24 00:00:00.0|74.95               

# **DATAFRAME FROM JSON FILE**

In [0]:
ordersDF = spark.read.json("/content/drive/My Drive/BIG DATA/pyspark/CCA-175/dataset/retail_db_json/orders")

In [0]:
ordersDF.printSchema()

In [0]:
ordersDF.first()

In [0]:
ordersDF.select("order_id", "order_date").show(10, False)

# **WINDOWING FUNCTIONS**

In [0]:
orderItemsDF = spark.read.json("/content/drive/My Drive/BIG DATA/pyspark/CCA-175/dataset/retail_db_json/order_items/part-r-00000-6b83977e-3f20-404b-9b5f-29376ab1419e")

In [0]:
orderItemsDF.printSchema()

root
 |-- order_item_id: long (nullable = true)
 |-- order_item_order_id: long (nullable = true)
 |-- order_item_product_id: long (nullable = true)
 |-- order_item_product_price: double (nullable = true)
 |-- order_item_quantity: long (nullable = true)
 |-- order_item_subtotal: double (nullable = true)



In [0]:
from pyspark.sql.window import *
from pyspark.sql.functions import min, max, avg

In [0]:
orders_orderItems_DF = ordersDF.filter(ordersDF.order_status.isin('COMPLETE', 'CLOSED')).join(order_itemsDF, on=ordersDF.order_id == order_itemsDF.order_item_order_id,how='inner')\
    .groupBy('order_date', 'order_item_product_id')\
    .agg(round(sum('order_item_subtotal'),2).alias('revenue'))\
    .sort('order_date', 'revenue',ascending=[True,False])
orders_orderItems_DF.show(truncate=False)

+---------------------+---------------------+-------+
|order_date           |order_item_product_id|revenue|
+---------------------+---------------------+-------+
|2013-07-25 00:00:00.0|1004                 |5599.72|
|2013-07-25 00:00:00.0|191                  |5099.49|
|2013-07-25 00:00:00.0|957                  |4499.7 |
|2013-07-25 00:00:00.0|365                  |3359.44|
|2013-07-25 00:00:00.0|1073                 |2999.85|
|2013-07-25 00:00:00.0|1014                 |2798.88|
|2013-07-25 00:00:00.0|403                  |1949.85|
|2013-07-25 00:00:00.0|502                  |1650.0 |
|2013-07-25 00:00:00.0|627                  |1079.73|
|2013-07-25 00:00:00.0|226                  |599.99 |
|2013-07-25 00:00:00.0|24                   |319.96 |
|2013-07-25 00:00:00.0|821                  |207.96 |
|2013-07-25 00:00:00.0|625                  |199.99 |
|2013-07-25 00:00:00.0|705                  |119.99 |
|2013-07-25 00:00:00.0|572                  |119.97 |
|2013-07-25 00:00:00.0|666  

In [0]:
orders_orderItems_DF = ordersDF.filter(ordersDF.order_status.isin('COMPLETE', 'CLOSED')).join(order_itemsDF, on=ordersDF.order_id == order_itemsDF.order_item_order_id,how='inner')
orders_orderItems_DF.select('order_date', 'order_item_product_id', 'order_item_subtotal')\
    .withColumn('daily_revenue', round(sum('order_item_subtotal').over(Window.partitionBy('order_date','order_item_product_id')),2))\
    .sort('order_date', 'daily_revenue',ascending=[True,False])\
    .select('order_date', 'order_item_product_id', 'daily_revenue')\
    .show(truncate=False)

+---------------------+---------------------+-------------+
|order_date           |order_item_product_id|daily_revenue|
+---------------------+---------------------+-------------+
|2013-07-25 00:00:00.0|1004                 |5599.72      |
|2013-07-25 00:00:00.0|1004                 |5599.72      |
|2013-07-25 00:00:00.0|1004                 |5599.72      |
|2013-07-25 00:00:00.0|1004                 |5599.72      |
|2013-07-25 00:00:00.0|1004                 |5599.72      |
|2013-07-25 00:00:00.0|1004                 |5599.72      |
|2013-07-25 00:00:00.0|1004                 |5599.72      |
|2013-07-25 00:00:00.0|1004                 |5599.72      |
|2013-07-25 00:00:00.0|1004                 |5599.72      |
|2013-07-25 00:00:00.0|1004                 |5599.72      |
|2013-07-25 00:00:00.0|1004                 |5599.72      |
|2013-07-25 00:00:00.0|1004                 |5599.72      |
|2013-07-25 00:00:00.0|1004                 |5599.72      |
|2013-07-25 00:00:00.0|1004             